In [43]:
#Importing packages we need
from numpy import genfromtxt #This is used to loading a csv-file as a numpy array
import numpy as np
import random
import scipy.optimize as opt #used to numerically optimize
import matplotlib.pyplot as plt #pyplot is used to plot the data
import pandas as pd
from matplotlib import cm
import numdifftools as nd #Using this package to numerically approximate the hessian matrix
import time #Used to measure time to run code
from scipy.stats import norm

In [44]:
#Defining all functions we need
def sim_option_price_sdf(S_t, K, horizon, par, draws,r):
    omega = par[0]
    alpha = par[1]
    beta = par[2]
    gamma = par[3]

    S_T = np.zeros(draws)
    temp = np.zeros(draws)
    
    T = len(x)
    
    #//sigma2 sequence until time t
    sigma2 = np.ones(T)
    #//mu sequence until time t
    mu = np.ones(T)
    
    sigma2[0] = np.var(x) #//Initial value. Other values could be chosen. Also the code could be modified such that the initial conditional volatility is a parameter to be estimated.
    mu[0]=gamma
    
    for t in range(1,T):
        mu[t]=gamma
        sigma2[t]=omega+alpha*(x[t-1]-mu[t-1])**2+beta*sigma2[t-1]
        
    #//simulation of price paths and paths of the SDF
    for i in range(draws):
        ret = np.zeros(horizon) #//return sequence
        theta =np.zeros(horizon) #//theta sequence
        sdf =np.zeros(horizon) #//sdf sequence
        m =np.zeros(horizon) #//mu sequence
        s = sigma2[-1] #//estimate of sigma_t^2 with t the time of the last observation
        s = omega + alpha*(x[-1]-mu[-1])**2 + beta*s  #//estimate of sigma_{t+1}^2 with t the time of the last observation
        for j in range(horizon):
            m[j]=gamma
            theta[j]=(r-m[j])/s-1/2 #//estimate of theta_t
            ret[j] =mu[j]+np.sqrt(s)*np.random.normal(loc=0.0, scale=1.0, size=1)[0] #simulated return
            sdf[j]= np.exp(theta[j]*ret[j])/np.exp((1+theta[j])*m[j]+(1+theta[j])**2*s/2) #simulated sdf, i.e. m_{t,t+1}
            s=omega + alpha*(ret[j]-mu[j])**2 + beta*s #//estimate of sigma_t+1^2 [i.e. sigma2 at time j+1]
        rsum=np.sum(ret) #//accumulated return over the horizon
        S_T[i] = S_t*np.exp(rsum) #//simulated price at the expiration date of the option (i.e. at the end of the horizon)
        prodsdf = np.prod(sdf)  #//simulated sdf for discounting payoffs occuring at the expiration date, i.e. m_{t,T}
        temp[i]=prodsdf*(S_T[i]-K)*int(S_T[i]>K) #//m_{t,T}*max(S_T-K,0)
    return np.mean(temp)
def black_scholes_h_periods(vol,K,S,r,h): #h = T-t , vol = sigma
    u=(np.log(K*np.exp(-(r-vol**2/2)*h))-np.log(S))/(vol*np.sqrt(h))
    return S*norm.cdf(-u+vol*np.sqrt(h))-np.exp(-r*h)*K*norm.cdf(-u)

def GARCH11_filter(par,x): #function to find sigma^2 from observations
    T=len(x)
    omega = par[0]
    alpha = par[1]
    beta = par[2]
    gamma = par[3]
    #Define the series sigma^2_t (sigma2) as empty variables we assign values to later. 
    sigma2=np.empty(T)
    mu=np.zeros(T)
    sigma2[:]=np.nan #making all values NaN so we can check for errors afterwards
    
    sigma2[0]=np.var(x)
    mu[0]=gamma
    #sigma2[0]=sigma_bar/(1-alpha-beta) - Should normally use this expression but when we optimize the sigma2 might turn negative leading to errors
    
    for t in range(1,T):
        mu[t]=gamma #//Note that the mu sequence could be something else. E.g. could include AR term. Parameters and expressions for mu have to be adjusted accordingly.
        sigma2[t]=omega + alpha*(x[t-1]-mu[t-1])**2+beta*sigma2[t-1]
    
    return sigma2,mu

def GARCH11_likelihood(par):
    
    sigma2,mu=GARCH11_filter(par,x)
    
    LogL=-np.sum(-np.log(sigma2)-(x-mu)**2/sigma2)
    
    return LogL

def optim(Par):
    par=Par
    return GARCH11_likelihood(par)

In [45]:
folder='Insert your path here! xD)'
data=np.genfromtxt(folder+'SP500.csv', delimiter=',',usecols=np.arange(0,4)) #loading in first 4 columns
x = data[15097:, 3:4]*100 #100 times log-returns of the S&P 500 index. January 4, 2010 - November 28, 2014
x=x.T[0,:] #unpacking numpy array

In [46]:
#GARCH Estimation
#Maximization
Par0=np.array([0.1,0.5,0.6,0]) #initial guesses
res=opt.minimize(optim, Par0, method='SLSQP',bounds=((0.0001,None),(0,None),(0,1),(None,None)))

In [47]:
par=res.x #Assigning results from optimization to "Par"
par

array([0.04136765, 0.14644764, 0.81185233, 0.07278178])

In [48]:
#//We rescale the returns the volatility has a correct scale
x = x/100
#//We rescale the estimate of gamma and omega such that the conditional mean and volatility have a correct scale
par[0] = par[0]/100**2 #//omega
par[3] = par[3]/100 #//gamma
r= 0.003/251 #// Risk free interest rate of 0.3 percent per year converted into a daily rate.	The interest rate is divided by 251, which is roughly the number of trading days during a year.
S_t = 1990.20# // Last traded price of the index on September 17, 2015
K= 1950 #// Strike price
h= 21#// Days until maturity of the option, i.e. T - t. 21 corresponds to the option expiring on October 16, 2015.
#//We rescale the returns the volatility has a correct scale
B=100000 #//number of draws for simulations
print('Price based on SDF, (Zhu and Ling, 2015) = '+str(sim_option_price_sdf(S_t, K, h, par, B,r)))

Price based on SDF, (Zhu and Ling, 2015) = 62.13782042836884


In [49]:
strikes=np.array([1650,1700,1750,1800,1850,1900,1950, 2000,2050 ,2100, 2150, 2200])
prices  = np.array([335.2, 286.2, 237.8, 190.5, 144.95, 102.1, 63.45, 31.3, 10.0, 1.6, 0.3, 0.1])
numpr=len(strikes) #number of prices we need to compute
pbs = np.zeros(numpr) #//array for prices based on BS formula
pgarch = np.zeros(numpr) #//array for prices based on GARCH
vol=np.var(x)**0.5

In [50]:
start_time = time.time()
for i in range(numpr):
    pbs[i]=black_scholes_h_periods(vol, strikes[i], S_t, r, h) # //price based on BS
    pgarch[i]=sim_option_price_sdf(S_t, strikes[i], h, par, B,r) #//price based on GARCH
print('Elapsed time (all computations)+'"--- %s seconds ---" % (time.time() - start_time))

Elapsed time (all computations)+--- 415.189071893692 seconds ---


In [51]:
Results=pd.DataFrame(strikes,columns=['Strikes']) #creating a dataframe to present results nicely

In [52]:
Results['Mkt. Prices']=prices
Results['BS Prices']=pbs
Results['GARCH Prices']=pgarch

In [53]:
Results

,Strikes,Mkt. Prices,BS Prices,GARCH Prices
0,1650,335.20,340.614516,340.744421
1,1700,286.20,290.633225,290.857165
2,1750,237.80,240.704948,241.369245
3,1800,190.50,191.094072,192.120780
4,1850,144.95,142.758165,144.416391
5,1900,102.10,97.958817,100.104088
6,1950,63.45,60.112253,62.281101
7,2000,31.30,32.185450,34.034541
8,2050,10.00,14.753750,16.391705
9,2100,1.60,5.720934,7.564411


In [54]:
print('average absolute relative pricing error, GARCH '+str(abs((Results['Mkt. Prices'][0:8]-Results['GARCH Prices'][0:8])/Results['Mkt. Prices'][0:8]).mean())) #ignoring out of money options i.e Strike ov 2100 and higher
print('average absolute relative pricing error, BS '+str(abs((Results['Mkt. Prices'][0:8]-Results['BS Prices'][0:8])/Results['Mkt. Prices'][0:8]).mean())) #ignoring out of money options i.e Strike ov 2100 and higher

average absolute relative pricing error, GARCH 0.023168541263506353
average absolute relative pricing error, BS 0.022944046697527375
